# Usage

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import scipy
import seaborn as sns
import tifffile
from pympler.asizeof import asizeof

from nima import io, nima, segmentation

cm = plt.cm.inferno_r

fp = "../../tests/data/1b_c16_15.tif"

channels = ["G", "R", "C"]
# dark = io.imread('/home/dati/GBM_persson/analyses/15.02.05_cal-GBM5-pBJclop/dark/dark-25_500.tif')
# flat = io.imread('/home/dati/GBM_persson/analyses/15.02.05_cal-GBM5-pBJclop/flat/flat-C-dark-37bis_500.tif')

In [ ]:
bg_params = segmentation.BgParams(kind="li_adaptive")
bg_params

In [ ]:
da = io.read_image(fp, channels)
da.metadata

In [ ]:
da.C

In [ ]:
ratio_ims = da.sel(C="G") / da.sel(C="C")

plt.imshow(ratio_ims[0][0])

In [ ]:
da.attrs.keys()

In [ ]:
da.metadata.voxel_size

In [ ]:
from bioio import BioImage

bi = BioImage("../../tests/data/1b_c16_15.tif")


xa = bi.get_xarray_dask_stack()

bi.physical_pixel_sizes.X

In [ ]:
xarray_dask_data = bi.xarray_dask_data.assign_coords(C=list(channels))

In [ ]:
bi.dims

In [ ]:
da.sizes["T"]

In [ ]:
bi.reader

In [ ]:
bi.resolution_level_dims

In [ ]:
bi.resolution_levels

In [ ]:
bi.scale

In [ ]:
bi.time_interval

In [ ]:
bi.standard_metadata.__dict__

In [ ]:
xarray_dask_data.data

In [ ]:
bi.xarray_dask_data.data

In [ ]:
f = nima.plot_img(da)

In [ ]:
im_da, bgs, ff = nima.bg(da, downscale=(2, 2), bg_params=bg_params)
bgs

In [ ]:
(
    da.sel(C="G").__sizeof__(),
    da.sel(C="G").data.__sizeof__(),
    da.sel(C="G").data.compute().__sizeof__(),
)

In [ ]:
ff["C"][1][0]

In [ ]:
labels_da = nima.segment(
    im_da, threshold_method="yen", min_size=2000, channels=channels, watershed=0
)

In [ ]:
f = nima.plot_img(labels_da, channels=[], labels=labels_da)

In [ ]:
meas, pr = nima.measure(da, labels_da)

In [ ]:
f = nima.plot_meas(bgs, meas, channels=channels)

In [ ]:
r_cl = nima.ratio(da, mask=labels_da > 0)
r_ph = nima.ratio(da, mask=labels_da.astype(bool), channels=["G", "C"])

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(r_cl[2], vmin=0.0, vmax=1.1)
plt.subplot(1, 2, 2)
plt.imshow(r_ph[0], vmin=7.3, vmax=10.7)

## dev

In [ ]:
c0 = da.sel(C="C")[0]
g0 = da.sel(C="G")[0]
r0 = da.sel(C="R")[0]
tifffile.imshow(1 / c0 / c0)

In [ ]:
bg_params = segmentation.BgParams(kind="li_adaptive", erosion_disk=0)
bg_params

In [ ]:
im = c0
im = da.sel(C="C")[1]
bg_params.adaptive_radius = int(im.shape[1] / 2)
if bg_params.adaptive_radius % 2 == 0:  # sk >0.12.0 check for even value
    bg_params.adaptive_radius += 1
bg_params

In [ ]:
im = im[0].data.compute()

In [ ]:
bg_params.clip = False
m, title, lim = segmentation._bg_li_adaptive(im, bg_params=bg_params)
title, lim

In [ ]:
scipy.stats.distributions.norm.fit(im[m])

In [ ]:
sns.histplot(im[m], kde=True, stat="density", log_scale=(False, True))

In [ ]:
fig = plt.hist(im[m], histtype="step", bins=32, log=1)
fig = plt.hist(im[m], histtype="step", bins=32, log=1)

In [ ]:
segmentation.calculate_bg_iteratively(im, probplot=True)

## background, AF, target cells

In [ ]:
fpm = "../../tests/data/1b_c16_15.tif"

In [ ]:
dim = io.read_image(fpm, channels)

dim.metadata

In [ ]:
dim = dim.squeeze("Z")
dim.shape

In [ ]:
bgd = {}
for i, c in enumerate(channels):
    bgd[c] = [
        segmentation.calculate_bg_iteratively(dim[t, i].data.compute())
        for t in range(dim.metadata.size_t[0])
    ]

bgd

In [ ]:
di = dim.astype(float)

In [ ]:
for t in range(dim.metadata.size_t[0]):
    di[t, 0] -= bgd[channels[0]][t].bg

In [ ]:
plt.imshow(di[0, 0])
plt.colorbar()

In [ ]:
g = di.sel(C="G").to_numpy()
g

In [ ]:
segmentation.calculate_bg_iteratively(g[0], probplot=1)

In [ ]:
bgd[channels[0]][1].bg

In [ ]:
t = 0

plt.imshow(dim[t, 0] - bgd[channels[0]][t].bg)
plt.colorbar()

In [ ]:
m = segmentation.prob(dim[2, 0].compute(), 0, bgd[channels[0]][2].sd * 13) > 0.001
plt.imshow(~m)

In [ ]:
asizeof(dim) / 1024**2

In [ ]:
fpm = "../../tests/data/1b_c16_15.tif"
tifffile.TiffFile(fpm).series

In [ ]:
from bioio import BioImage

md = io.Metadata(BioImage(fpm).metadata)
md

In [ ]:
asizeof(dim) / 1024**2

In [ ]:
bgr = segmentation.calculate_bg_iteratively(dim[1, 0, 0].data.compute())
bgr.bg

In [ ]:
dim3 = io.read_image(fpm, channels)

In [ ]:
dim3, bgv3, bgf3 = nima.bg(
    dim3, segmentation.BgParams(kind="li_adaptive"), downscale=(2, 2)
)
c = dim3.sel(C="C")
g = dim3.sel(C="G")
r = dim3.sel(C="R")

In [ ]:
bc = bgv3["C"]
bg = bgv3["G"]
br = bgv3["R"]

In [ ]:
plt.imshow(dim3.sel(C="R").data.compute()[1][0])

In [ ]:
plt.plot(dim3.sel(C="R")[1, 0][80, 30:])

In [ ]:
green = dim.sel(C="G").data.compute()
cyan = dim.sel(C="C").data.compute()
red = dim.sel(C="R").data.compute()
green

In [ ]:
# green = dim[0,0,0].data.compute()
bgg = segmentation.calculate_bg_iteratively(green)
bgg

In [ ]:
mask = segmentation.prob(green, bgg.bg, bgg.sd) > 0.01
green[mask]

In [ ]:
# cyan = dim[2].compute()
bgc = segmentation.calculate_bg_iteratively(cyan)
mask = segmentation.prob(cyan, bgc.bg, bgc.sd) < 0.00000000000000000000001

In [ ]:
plt.imshow(mask[3])

In [ ]:
# red = dim[2].compute()
# red2 = dim[1].compute()
red2 = red

In [ ]:
plt.figure(figsize=(7.5, 7.5))

plt.subplot(3, 2, 1)
plt.hexbin(green[mask], cyan[mask], bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("green")
plt.ylabel("cyan")

plt.subplot(3, 2, 2)
plt.hexbin(red[mask], cyan[mask], bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("red")
plt.ylabel("cyan")

ax = plt.subplot(3, 2, 4)
plt.hexbin(red[mask], green[mask], bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("red")
plt.ylabel("green")

ax = plt.subplot(3, 2, 5)
plt.hexbin(red2[mask], green[mask], bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("red2")
plt.ylabel("green")

ax = plt.subplot(3, 2, 6)
plt.hexbin(red2[mask], cyan[mask], bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("red2")
plt.ylabel("cyan")

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

plt.figure(figsize=(10, 8))
plt.subplot(2, 2, 1)
# plt.hexbin(g.ravel(), c.ravel(), bins="log", cmap=plt.cm.viridis_r)
plt.hexbin(g, c, bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("green")
plt.ylabel("cyan")

plt.subplot(2, 2, 2)
plt.hexbin(r, c, bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("red")
plt.ylabel("cyan")

ax = plt.subplot(2, 2, 4)
plt.hexbin(r, g, bins="log", cmap=plt.cm.viridis_r)
cb = plt.colorbar()
plt.xlabel("red")
plt.ylabel("green")

axins = plt.axes([0.2, 0.12, 0.28, 0.28])
axins.hexbin(r, g, extent=(0, 80, 0, 150), bins="log", cmap=plt.cm.viridis_r)

mark_inset(ax, axins, loc1=1, loc2=4, fc="none", ec="0.5")

In [ ]:
import statsmodels.api as sm

f = sm.qqplot_2samples(red[~mask], green[~mask])
f = sm.qqplot_2samples(red[~mask], cyan[~mask])

## flat image correction

## example w/out @Gain

## BIAS and FLAT

TODO:
- pytest
- build a function callable from both the library and the CLI